In [1]:
import collections

ROOT_DIR = "/Users/anca/Desktop/advent_code/advent_code2023/day_7/"


def load_map(path):
    schema = open(path).readlines()
    # print(schema)
    return [line.rstrip() for line in schema]


cards = load_map(f"{ROOT_DIR}cards.txt")
small_hand = load_map(f"{ROOT_DIR}small_hand.txt")
# boats_example = load_map(f'{ROOT_DIR}example.txt')
print(cards)

['6JA22 162', 'TQJQ8 732', '7T77A 882', '6K66K 850', 'QQAQQ 11', '7QQ7Q 321', '28966 921', '34433 801', '8QQ8Q 914', 'K63K5 636', '47777 533', 'K62Q9 773', '25J52 717', 'A6AAA 631', '23222 720', '267Q4 9', 'K8QK5 15', 'QQA22 400', '563T8 50', '2J5J2 756', '77T83 647', 'T9596 27', '53AJA 371', '6J666 916', '833TK 380', '5TK4J 506', 'TK3KK 794', 'JTJ55 54', 'TTT4T 422', '9J3KK 491', '488K4 30', '87878 654', 'QQKKK 894', 'A4A3J 360', '55559 232', '33388 148', 'JK599 632', 'KK663 300', 'JJ448 514', 'AKATA 188', '6J537 885', '98888 65', '9TT3J 361', '668A9 227', 'Q3285 257', 'JT54J 208', 'K8QA7 373', '77K39 681', '33T4J 281', 'KA832 578', '6QQQ5 214', '52222 154', '6A4Q4 78', '39339 282', '2A82A 25', '5A66J 261', '5594J 4', 'TTTT6 752', '9A3K5 760', 'KA5AA 26', 'K7464 447', '33676 605', 'K7J7A 946', '5T5KQ 805', '654Q8 888', '35358 315', 'A4A57 552', '669AA 485', '7A77Q 105', 'Q3333 372', '57A9Q 665', '97655 367', '24222 698', 'Q6KT7 820', '99KKK 83', 'T33JK 389', '9KKQ2 602', 'TT9JQ 278', 

In [2]:
import requests

# log in
login_url = "https://github.com/login/oauth/authorize?client_id=b0b9e4e723fdb8841400"
login_page = requests.get(login_url)
# print(login_page.text)
# get puzzle input
URL = "https://adventofcode.com/2023/day/7/input"
INPUT = requests.get(URL).text
# print(INPUT)

In [6]:
"""
Build a class to represent the cards
- attributes: Number
- methods:
    - get_number
    - __lt__

"""

from enum import Enum


class Rule(Enum):
    """
    Defines the rules of the game.
    CLASSIC: part a
    JOKER: part b
    """

    CLASSIC = 0
    JOKER = 1


class Card:
    """
    Class to represent a card
    Changes the ranking of the cards depending on the rules of the game
    """

    def __init__(self, number, rule=Rule.CLASSIC):
        self.card_number_to_score = {
            "A": 15,
            "K": 14,
            "Q": 13,
            "J": 12,
            "T": 10,
        }
        if rule == Rule.JOKER:
            self.card_number_to_score["J"] = 1
        self.card = number
        self.number = None
        if number in self.card_number_to_score:
            self.number = self.card_number_to_score[number]
        else:
            self.number = int(number)

    def __lt__(self, other):
        return self.number < other.number

    def __gt__(self, other):
        return self.number > other.number

    def __eq__(self, other):
        return self.number == other.number

    def __repr__(self):
        return f"{self.card}"


ace = Card("A")
ten = Card("T")
five = Card("5")
assert ace > ten
assert ten > five
assert five < ace
print(ace, ten, five)

# check we can instantiate the cards with the new rules
joker = Card("J", Rule.JOKER)
J = Card("J")
assert joker < J

A T 5


In [8]:
"""
Build a class to represent a hand of cards and compare hands.
The hand types are: high card, one pair, two pairs, three of a kind, full house, four of a kind,
five of a kind. The hand with the highest type wins. If the types are the same, we compare hands
card by card until we find a difference. The hand with the highest card"""

from enum import Enum


class HandType(Enum):
    HIGH_CARD = 0
    ONE_PAIR = 1
    TWO_PAIRS = 2
    THREE_OF_A_KIND = 3
    FULL_HOUSE = 4
    FOUR_OF_A_KIND = 5
    FIVE_OF_A_KIND = 6

    def __lt__(self, other):
        if isinstance(other, HandType):
            return self.value < other.value
        return NotImplemented


class Hand:
    def __init__(self, cards, bid=0, rule=Rule.CLASSIC):
        # hold cards as a dictionary of numbers
        self.rule = rule
        self.cards = [Card(card, rule=self.rule) for card in cards]
        self.card_dic = self._get_card_count()
        self.type = self._get_hand_type()
        self.bid = int(bid)

    def _get_card_count(self):
        """Return a dictionary with the count of each card"""
        card_dic = collections.defaultdict(int)
        for card in self.cards:
            card_dic[card.card] += 1
        return card_dic

    def _get_hand_type(self):
        """Return the type of the hand as a HandType enum value"""

        def _is_k_of_a_kind(k, rule=self.rule):
            """
            Return True if the hand has k cards of the same value
            Add the option to set the rule manually to enable more complex computations such
            as determining full house with jokers
            """

            num_jokers = self.card_dic.get("J", 0)
            for card_count in self.card_dic.values():
                # count jokers in the hand if the rule is JOKER
                if rule == Rule.JOKER:
                    if card_count + num_jokers == k:
                        return True
                # do a regular count if rule is CLASSIC
                if card_count == k:
                    return True
            return False

        def _is_two_pairs(rule=self.rule):
            """
            Return True if the hand has two pairs of cards
            """
            num_pairs = 0
            for card_count in self.card_dic.values():
                if card_count == 2:
                    num_pairs += 1
            # check if the hand has two pairs in the case of JOKER rule
            if rule == Rule.JOKER:
                num_jokers = self.card_dic.get("J", 0)
                if num_jokers == 2:
                    return True
                if num_jokers == 1 and num_pairs == 1:
                    return True
            # check if the hand has two pairs in the case of CLASSIC rule
            return num_pairs == 2

        # check if the hand is five of a kind
        if _is_k_of_a_kind(5):
            return HandType.FIVE_OF_A_KIND
        # check if the hand is four of a kind
        if _is_k_of_a_kind(4):
            return HandType.FOUR_OF_A_KIND
        # check if the hand is full house
        if self.rule == Rule.JOKER and "J" in self.card_dic:
            num_jokers = self.card_dic.get("J", 0)
            # num_jokers >= 2 should not happen bc. we would have a better hand
            # if we have 1 joker we need 3 of a kind to make full house or 2 pairs
            if (
                num_jokers == 1
                and _is_k_of_a_kind(3, rule=Rule.CLASSIC)
                or _is_two_pairs(rule=Rule.CLASSIC)
            ):
                return HandType.FULL_HOUSE
            else:
                if _is_k_of_a_kind(3):
                    return HandType.THREE_OF_A_KIND
        else:
            if _is_k_of_a_kind(3) and _is_k_of_a_kind(2):
                return HandType.FULL_HOUSE
        # check if the hand is 3 of a kind
        if _is_k_of_a_kind(3):
            return HandType.THREE_OF_A_KIND
        # check if the hand is two pairs
        if _is_two_pairs():
            return HandType.TWO_PAIRS
        # check if the hand is one pair
        if _is_k_of_a_kind(2):
            return HandType.ONE_PAIR
        return HandType.HIGH_CARD

    def __lt__(self, other):
        """Compare two hands. Used for powering the < operator and sorting hands."""

        if self.type == other.type:
            for i in range(len(self.cards)):
                if self.cards[i] != other.cards[i]:
                    return self.cards[i] < other.cards[i]
        return self.type < other.type

    def to_string(self):
        """Return a string representation of the hand"""
        return f"{self.cards} - {self.type} - {self.bid} - {self.rule}"


# test the Hand class
high_card = Hand([1, 2, 3, 4, 5], rule=Rule.JOKER)
print(high_card.to_string())
assert high_card.type == HandType.HIGH_CARD
assert [card.number for card in high_card.cards] == [1, 2, 3, 4, 5]

# check that the best hand is identified. Full house vs. 3 of a kind
full_house = Hand([2, 3, 2, 2, 3], rule=Rule.CLASSIC)
assert full_house.type == HandType.FULL_HOUSE

# check that 2 pairs is identifies
two_pairs = Hand([2, 6, 6, 2, 4], rule=Rule.CLASSIC)
assert two_pairs.type == HandType.TWO_PAIRS

# test sorting for different hand types
assert high_card < full_house
assert full_house > two_pairs
assert high_card < two_pairs

# test sorting for the same hand type
another_two_pairs = Hand([2, 6, 6, 2, 5])
assert two_pairs < another_two_pairs

another_full_house = Hand([9, 9, 2, 2, 2])
assert full_house < another_full_house

# test the hand type with joker rules
three_of_a_kind = Hand([2, 2, "J", 3, 4], rule=Rule.JOKER)
print(three_of_a_kind.to_string())
assert three_of_a_kind.type == HandType.THREE_OF_A_KIND

three_of_a_kind = Hand([2, 5, "J", 3, 3], rule=Rule.JOKER)
assert three_of_a_kind.type == HandType.THREE_OF_A_KIND

two_pairs = Hand([2, "J", 3, 2, 3], rule=Rule.JOKER)
assert two_pairs.type == HandType.FULL_HOUSE

[1, 2, 3, 4, 5] - HandType.HIGH_CARD - 0 - Rule.JOKER
[2, 2, J, 3, 4] - HandType.THREE_OF_A_KIND - 0 - Rule.JOKER


In [27]:
# extract the hands from the input
def get_score(cards, rule=Rule.CLASSIC) -> int:
    """
    Takes in a list of hands as a text input and returns the score of the game.
    The score is computed by multiplying the bid of each hand by its position in the sorted list.
    """
    hands = []
    for line in cards:
        hand, score = line.split()
        hands.append(Hand(hand, bid=score, rule=rule))
    # print([hand.to_string() for hand in hands])
    # sort the hands
    hands.sort(reverse=True)
    print(
        [
            hand.to_string()
            for hand in hands
            if hand.type
            in [HandType.TWO_PAIRS, HandType.ONE_PAIR]
        ]
    )

    # compute the score
    score = 0
    multiplier = len(hands)
    for hand in hands:
        score += hand.bid * multiplier
        multiplier -= 1
    return score


# For part A, the score is computed with the classic rules
print("Part A")
print(f"cards: {cards}")
print(f"score of all hands {get_score(cards)}")
print(f"example cards {small_hand}")
print(f"score of example {get_score(small_hand)}")

Part A
cards: ['6JA22 162', 'TQJQ8 732', '7T77A 882', '6K66K 850', 'QQAQQ 11', '7QQ7Q 321', '28966 921', '34433 801', '8QQ8Q 914', 'K63K5 636', '47777 533', 'K62Q9 773', '25J52 717', 'A6AAA 631', '23222 720', '267Q4 9', 'K8QK5 15', 'QQA22 400', '563T8 50', '2J5J2 756', '77T83 647', 'T9596 27', '53AJA 371', '6J666 916', '833TK 380', '5TK4J 506', 'TK3KK 794', 'JTJ55 54', 'TTT4T 422', '9J3KK 491', '488K4 30', '87878 654', 'QQKKK 894', 'A4A3J 360', '55559 232', '33388 148', 'JK599 632', 'KK663 300', 'JJ448 514', 'AKATA 188', '6J537 885', '98888 65', '9TT3J 361', '668A9 227', 'Q3285 257', 'JT54J 208', 'K8QA7 373', '77K39 681', '33T4J 281', 'KA832 578', '6QQQ5 214', '52222 154', '6A4Q4 78', '39339 282', '2A82A 25', '5A66J 261', '5594J 4', 'TTTT6 752', '9A3K5 760', 'KA5AA 26', 'K7464 447', '33676 605', 'K7J7A 946', '5T5KQ 805', '654Q8 888', '35358 315', 'A4A57 552', '669AA 485', '7A77Q 105', 'Q3333 372', '57A9Q 665', '97655 367', '24222 698', 'Q6KT7 820', '99KKK 83', 'T33JK 389', '9KKQ2 602',

In [22]:
hand1 = Hand(["J", "J", "J", "J", "J"], rule=Rule.JOKER)
hand2 = Hand(["J", "J", "J", 8, "J"], rule=Rule.JOKER)
hand3 = Hand(["A", "A", 7, 9, 7], rule=Rule.JOKER)
hand4 = Hand(["A", "A", 7, 7, 7], rule=Rule.JOKER)
hand5 =  Hand(["A", "J", 7, 7, 7], rule=Rule.JOKER)

assert Hand(["A", "J", 7, 7, 7], rule=Rule.JOKER).type == HandType.FOUR_OF_A_KIND
assert Hand(["A", "A", "J", 7, 7], rule=Rule.JOKER).type == HandType.FULL_HOUSE
assert Hand(["A", "6", "J", 7, 7], rule=Rule.JOKER).type == HandType.THREE_OF_A_KIND


print(hand3._get_hand_type())
print(hand2._get_hand_type())
print(hand1._get_hand_type())
print(hand4._get_hand_type())

assert hand1 < hand2




HandType.TWO_PAIRS
HandType.FIVE_OF_A_KIND
HandType.FIVE_OF_A_KIND
HandType.FULL_HOUSE


### Part 2

* Now J cards are jokers. They are only worth 2 points. They can take the shape of any card to make the best possible hand.

Steps:
* add dependency inject of rules to hand class. Change how we compute hands based on rules
* add a new Card method inheriting from the old one to  compute the value of a card based on the rules

In [29]:
# For part B, the score is computed with the joker rules
print("Part B")
print(f"score of all hands {get_score(cards, rule=Rule.JOKER)}")
print(f"score of example {get_score(small_hand, rule=Rule.JOKER)}")
# 252234980 =? too low


Part B
['[A, A, 7, 9, 7] - HandType.TWO_PAIRS - 268 - Rule.JOKER', '[A, A, 5, T, T] - HandType.TWO_PAIRS - 937 - Rule.JOKER', '[A, A, 4, 4, 2] - HandType.TWO_PAIRS - 572 - Rule.JOKER', '[A, 9, 8, 9, 8] - HandType.TWO_PAIRS - 703 - Rule.JOKER', '[A, 8, 4, A, 4] - HandType.TWO_PAIRS - 222 - Rule.JOKER', '[A, 3, Q, A, Q] - HandType.TWO_PAIRS - 72 - Rule.JOKER', '[A, 3, 6, 6, 3] - HandType.TWO_PAIRS - 24 - Rule.JOKER', '[A, 2, 2, T, T] - HandType.TWO_PAIRS - 996 - Rule.JOKER', '[K, K, Q, Q, 3] - HandType.TWO_PAIRS - 213 - Rule.JOKER', '[K, K, 8, 8, 3] - HandType.TWO_PAIRS - 417 - Rule.JOKER', '[K, K, 7, A, 7] - HandType.TWO_PAIRS - 73 - Rule.JOKER', '[K, K, 6, 6, T] - HandType.TWO_PAIRS - 672 - Rule.JOKER', '[K, K, 6, 6, 3] - HandType.TWO_PAIRS - 300 - Rule.JOKER', '[K, K, 5, 5, T] - HandType.TWO_PAIRS - 536 - Rule.JOKER', '[K, K, 3, 9, 9] - HandType.TWO_PAIRS - 948 - Rule.JOKER', '[K, K, 2, 8, 8] - HandType.TWO_PAIRS - 540 - Rule.JOKER', '[K, K, 2, 3, 3] - HandType.TWO_PAIRS - 962 - Rule.